In [ ]:
elastic_rods_dir = '../elastic_rods/python/'
weaving_dir = './'
import os
import os.path as osp
import sys; sys.path.append(elastic_rods_dir); sys.path.append(weaving_dir)
import numpy as np, elastic_rods, linkage_vis
import numpy.linalg as la
from bending_validation import suppress_stdout as so
import matplotlib.pyplot as plt
from elastic_rods import EnergyType, InterleavingType

# weaving
import analysis_helper, ribbon_linkage_helper, mesh_vis, linkage_utils, compute_curve_from_curvature, pipeline_helper, optimization_visualization_helper, importlib
importlib.reload(analysis_helper)
importlib.reload(ribbon_linkage_helper)
importlib.reload(mesh_vis)
importlib.reload(linkage_utils)
importlib.reload(compute_curve_from_curvature)
importlib.reload(pipeline_helper)
importlib.reload(optimization_visualization_helper)
from analysis_helper import (compare_turning_angle,
                            is_on_sphere, 
                            get_distance_to_center_scalar_field, 
                            plot_curvatures, 
                            get_curvature_scalar_field,
                            construct_elastic_rod_loop_from_rod_segments, 
                            concatenate_rod_properties_from_rod_segments, 
                            compute_min_distance_rigid_transformation)
from ribbon_linkage_helper import (update_rest_curvature, 
                                   set_ribbon_linkage,
                                   export_linkage_geometry_to_obj,
                                   write_linkage_ribbon_output_florin)

from compute_curve_from_curvature import (match_geo_curvature_and_edge_len, get_all_curve_pattern)
from linkage_utils import order_segments_by_ribbons, get_turning_angle_and_length_from_ordered_rods

from pipeline_helper import (initialize_linkage, get_normal_deviation, set_joint_vector_field, stage_1_optimization, initialize_stage_2_optimizer, stage_2_optimization, InputOrganizer, write_all_output, set_surface_view_options, get_structure_analysis_view, contact_optimization, get_double_side_view, show_selected_joints, highlight_rod_and_joint, write_distance_to_linkage_mesh, stage_1_optimization)
import vis.fields
import matplotlib.cm as cm
import time
from optimization_visualization_helper import (compute_visualization_data_from_raw_data, get_objective_components_stage1, get_objective_components_stage2, get_objective_components_stage3, set_figure_label_and_limit, Visualization_Setting, plot_objective, plot_ribbon_component_analysis, render_video, combine_three_stage_video)


In [ ]:
import parallelism
parallelism.set_max_num_tbb_threads(12)
parallelism.set_hessian_assembly_num_threads(4)
parallelism.set_gradient_assembly_num_threads(4)

In [ ]:
rod_length = 0.1756505993328395
width = rod_length / 15 * 5
thickness = width / 5 * 0.35
print(width, thickness)
io = InputOrganizer('regular_torus_1', thickness, width, weaving_dir)
#  [ ear 396, ear 412, nose 497]

In [ ]:
import py_newton_optimizer
OPTS = py_newton_optimizer.NewtonOptimizerOptions()
OPTS.gradTol = 1e-8
OPTS.verbose = 1;
OPTS.beta = 1e-8
OPTS.niter = 200
OPTS.verboseNonPosDef = False
rw = 1
sw = 10
drw = 0.1
dsw = 0.1

In [ ]:
import pickle 
import gzip

In [ ]:
curved_linkage = pickle.load(gzip.open('mega_monster_optimization_diagram_results/fifth_round/regular_torus_1/regular_torus_1_stage_2.pkl.gz', 'r'))
curved_linkage_view = linkage_vis.LinkageViewer(curved_linkage)
# set_surface_view_options(curved_linkage_view, linkage_color='lightgreen')
curved_linkage_view.show()

In [ ]:
curved_linkage.attraction_weight = 1e-5
def eqm_callback(prob, i):
    pass
elastic_rods.compute_equilibrium(curved_linkage, callback = eqm_callback, options = OPTS)
curved_linkage_view.update()

In [ ]:
E0 = curved_linkage.designParameterSolve_energy()
l0 = curved_linkage.totalRestLength()

In [ ]:
iterateData, _ = stage_1_optimization(curved_linkage, 0, 0.01, curved_linkage_view, E0 = E0, l0 = l0, niter = 200)

In [ ]:
import pickle
import gzip
pickle.dump(curved_linkage, gzip.open('{}_stage_2_20210108_crazy.pkl.gz'.format(io.MODEL_NAME), 'w'))

In [ ]:
# structure_view = get_structure_analysis_view(curved_linkage)
# structure_view.show()

In [ ]:
from structural_analysis import weavingCrossingForceMagnitudes
cfm = weavingCrossingForceMagnitudes(curved_linkage, True)
print("Normal Activation Threshold: ", np.percentile(cfm[:, 0], 75))

### Figure 9

Vector Plot

In [ ]:
save_path = os.getcwd()
write_all_output(curved_linkage, io.SUBDIVISION_RESOLUTION, io.MODEL_NAME + '_stage_3', io.RIBBON_NAME + '_stage_3', False, 100, min(io.RIBBON_CS), target_width = 5, flip_angles = False, width = max(io.RIBBON_CS))
os.chdir(save_path)